In [ ]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/2025-2. 홍콩대 학술교류/말뭉치 자료"

2022구어말뭉치	2023구어말뭉치	2023신문말뭉치	NIKL_SPOKEN_v1.2_JSON


In [ ]:
# 우리가 연구할 단어를 적으세요.

WORD_WE_USE = "싸다"
WORD_WE_USE_RE = "싸|쌀|싼|쌈|쌌|쌉" # 어간의 활용형 다 적기

In [ ]:
import json
import os

# 압축된 거 풀고 넣는 중
file_path_news = "/content/drive/MyDrive/2025-2. 홍콩대 학술교류/말뭉치 자료/2023신문말뭉치"
file_path_talk = "/content/drive/MyDrive/2025-2. 홍콩대 학술교류/말뭉치 자료/2023구어말뭉치"

news_files = [f for f in os.listdir(file_path_news) if f.endswith(".json")]
talk_files = [f for f in os.listdir(file_path_talk) if f.endswith(".json")]

print(f"신문 : 총 {len(news_files)}개의 JSON 파일이 있습니다.\n")
print(f"구어 : 총 {len(talk_files)}개의 JSON 파일이 있습니다.\n")

신문 : 총 28개의 JSON 파일이 있습니다.

구어 : 총 1985개의 JSON 파일이 있습니다.



# '차다'의 활용형만 추출

In [ ]:
import json
import os
from tqdm import tqdm
import re

def extract_forms(json_file, file_path, data_type):
    """
    지정된 JSON 파일에서 'form' 데이터를 추출하는 함수
    """
    file_full_path = os.path.join(file_path, json_file)

    try:
        with open(file_full_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        sentences_dict = {}

        # 신문과 구어의 구조에 따라 적절한 경로 선택
        if "document" in data:
            for doc in data["document"]:
                if data_type == "talk":  # 구어 말뭉치 구조
                    if "utterance" in doc:
                        for utterance in doc["utterance"]:
                            if "form" in utterance and utterance["form"]:
                                sentences_dict[utterance["id"]] = utterance["form"]
                elif data_type == "news":  # 신문 말뭉치 구조
                    if "paragraph" in doc:
                        for paragraph in doc["paragraph"]:
                            if "form" in paragraph and paragraph["form"]:
                                sentences_dict[paragraph["id"]] = paragraph["form"]

        return sentences_dict

    except Exception as e:
        print(f"❌ 파일 '{json_file}' 읽기 오류: {e}")
        return {}

def collect_sentences_with_words(file_path, search_pattern, data_type):
    """
    특정 패턴으로 시작하는 단어를 포함하는 모든 문장을 수집하는 함수
    """
    files = [f for f in os.listdir(file_path) if f.endswith(".json")]
    result = {}
    pattern = re.compile(rf"\b({search_pattern})[가-힣]*\b")  # 단어가 정확히 시작하고 끝나는 경우만 검색

    for file in tqdm(files, desc=f"{data_type} 파일 처리 중", unit="파일"):
        sentences_dict = extract_forms(file, file_path, data_type)
        filtered_sentences = {id_: sentence for id_, sentence in sentences_dict.items() if pattern.search(sentence)}

        if filtered_sentences:
            result.update(filtered_sentences)

    return result

In [ ]:
# 말뭉치 경로 (사용자가 지정 가능)
file_path_news_2023 = "/content/drive/MyDrive/2025-2. 홍콩대 학술교류/말뭉치 자료/2023신문말뭉치"  # 신문 말뭉치 경로
file_path_talk_2022 = "/content/drive/MyDrive/2025-2. 홍콩대 학술교류/말뭉치 자료/2022구어말뭉치"  # 구어 말뭉치 경로
file_path_talk_2023 = "/content/drive/MyDrive/2025-2. 홍콩대 학술교류/말뭉치 자료/2023구어말뭉치"  # 구어 말뭉치 경로

In [ ]:
# 정규식 검색 패턴
search_pattern = WORD_WE_USE_RE

# 문장 수집 실행
collected_2022_talk_sentences = collect_sentences_with_words(file_path_talk_2022, search_pattern, data_type="talk")
collected_2023_talk_sentences = collect_sentences_with_words(file_path_talk_2023, search_pattern, data_type="talk")
collected_2023_news_sentences = collect_sentences_with_words(file_path_news_2023, search_pattern, data_type="news")

# 결과 출력
print("----------------------------")
print(f"구어 말뭉치에서 수집된 문장: {len(collected_2022_talk_sentences)}개")
print(f"구어 말뭉치에서 수집된 문장: {len(collected_2023_talk_sentences)}개")
print(f"신문 말뭉치에서 수집된 문장: {len(collected_2023_news_sentences)}개")

news 파일 처리 중: 100%|██████████| 28/28 [03:23<00:00,  7.28s/파일]

----------------------------
구어 말뭉치에서 수집된 문장: 3183개
구어 말뭉치에서 수집된 문장: 2142개
신문 말뭉치에서 수집된 문장: 44729개


In [ ]:
if collected_2022_talk_sentences:
    print("\n📂 2022 구어 말뭉치 예제")
    for i, (id_, sentence) in enumerate(list(collected_2022_talk_sentences.items())[:10]):  # 처음 10개만 출력
        print(f"{i+1}. ID: {id_} | 문장: {sentence}")

if collected_2023_talk_sentences:
    print("\n📂 2023 구어 말뭉치 예제")
    for i, (id_, sentence) in enumerate(list(collected_2023_talk_sentences.items())[:10]):  # 처음 10개만 출력
        print(f"{i+1}. ID: {id_} | 문장: {sentence}")

if collected_2023_news_sentences:
    print("\n📂 2023 신문 말뭉치 예제")
    for i, (id_, sentence) in enumerate(list(collected_2023_news_sentences.items())[:10]):  # 처음 10개만 출력
        print(f"{i+1}. ID: {id_} | 문장: {sentence}")


📂 2022 구어 말뭉치 예제
1. ID: SDRW2200001995.1.1.269 | 문장: 도시락 싸서 챙겨 주고
2. ID: SDRW2200002000.1.1.51 | 문장: 나는 친구들이랑 싸우거나 그런 적은 없는데 친구들이 나를 미워한 적도 있었고
3. ID: SDRW2200002000.1.1.56 | 문장: 막 자기들끼리 막 싸워 가지고 막 엄청 소리 지르고 난리 치고
4. ID: SDRW2200002018.1.1.64 | 문장: 캄보디아 가서 나 싸웠거든.
5. ID: SDRW2200001996.1.1.261 | 문장: 아니 오히려 싸움을 걸었 싸움이 났는데
6. ID: SDRW2200002020.1.1.222 | 문장: 단백질 보충하기 위해서 소고기 사 먹는 거보다는 싸다.
7. ID: SDRW2200002020.1.1.239 | 문장: 아니 굴 자체가 어 우리나라에는 굉장히 싸지만은
8. ID: SDRW2200002029.1.1.174 | 문장: 싸워야 된다. 이건 좀 불공평한 것
9. ID: SDRW2200002024.1.1.215 | 문장: 그래서 지금 난리야 지금 서로 막 싸움이 일어나 가지고
10. ID: SDRW2200002025.1.1.84 | 문장: 혹시 도시락 싸 갖고 다니셨습니까?

📂 2023 구어 말뭉치 예제
1. ID: SDRW2300000979.1.1.41 | 문장: 더 이상 싸울 힘이
2. ID: SDRW2300000979.1.1.44 | 문장: 어 매번 싸우기만 했던 그 시간들이
3. ID: SDRW2300000984.1.1.264 | 문장: 마냥 선악의 싸움이다라고 보기만은 좀
4. ID: SDRW2300000998.1.1.137 | 문장: 그런 외국 쌀이 그렇잖아요.
5. ID: SDRW2300000998.1.1.138 | 문장: 그 그런 쌀 이렇게 요리 방법도 다르고
6. ID: SDRW2300000998.1.1.244 | 문장: 그래서 도시락을 싸 갔어요.
7. ID: SDRW2300000998.1.1.247

# 형태소 분석기 이용

In [ ]:
!pip -q install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 91.9 MB/s eta 0:00:00


In [ ]:
from kiwipiepy import Kiwi
from tqdm import tqdm

# Kiwi 형태소 분석기 초기화
kiwi = Kiwi()

def highlight_tada_sentences(sentences_dict):
    f"""
    문장에서 '{WORD_WE_USE[0]}'가 동사(VV) 또는 형용사(VA)로 쓰인 경우에만
    해당 단어를 강조 표시하여 결과에 포함시키는 함수.
    """
    highlighted_sentences = {}

    for id_, sentence in tqdm(sentences_dict.items(), desc="형태소 분석 진행 중", unit="문장"):
        analyzed = kiwi.analyze(sentence)
        tokens_to_highlight = []

        # 토큰 중 조건에 맞는 것만 선별
        for token in analyzed[0][0]:
            if token.form == f"{WORD_WE_USE[0]}" and token.tag in {"VV", "VA"}:
                tokens_to_highlight.append(token)

        # 조건에 맞는 토큰이 있을 때만 문자열 리스트 변환 및 수정 수행
        if tokens_to_highlight:
            char_list = list(sentence)
            for token in tokens_to_highlight:
                start, length = token.start, token.len
                char_list[start] = f"[{char_list[start]}"
                char_list[start + length - 1] = f"{char_list[start + length - 1]}]"
            highlighted_sentences[id_] = "".join(char_list)

    return highlighted_sentences

In [ ]:
import random

def sample_sentences(sentences_dict, sample_size=100000, random_state=42):
    """
    주어진 문장 딕셔너리에서 sample_size만큼 랜덤 샘플링하여 반환하는 함수.
    random_state가 주어지면, 재현 가능한 샘플링을 위해 해당 시드를 사용합니다.
    """
    rng = random.Random(random_state) if random_state is not None else random
    sampled_keys = rng.sample(list(sentences_dict.keys()), min(sample_size, len(sentences_dict)))
    return {key: sentences_dict[key] for key in sampled_keys}

In [ ]:
sampled_2023_news_sentences = sample_sentences(collected_2023_news_sentences)

# 어간의 활용형이 포함된 문장 필터링
filtered_2022_talk_sentences = highlight_tada_sentences(collected_2022_talk_sentences)
filtered_2023_talk_sentences = highlight_tada_sentences(collected_2023_talk_sentences)
filtered_2023_news_sentences = highlight_tada_sentences(sampled_2023_news_sentences)

# 결과 출력
print("------------------------------")
print(f"2022 구어 말뭉치에서 '{WORD_WE_USE}' 활용형이 포함된 문장: {len(filtered_2022_talk_sentences)}개")
print(f"2023 구어 말뭉치에서 '{WORD_WE_USE}' 활용형이 포함된 문장: {len(filtered_2023_talk_sentences)}개")
print(f"2023 신문 말뭉치에서 '{WORD_WE_USE}' 활용형이 포함된 문장: {len(filtered_2023_news_sentences)}개")

형태소 분석 진행 중: 100%|██████████| 44729/44729 [04:33<00:00, 163.73문장/s]

------------------------------
2022 구어 말뭉치에서 '싸다' 활용형이 포함된 문장: 1692개
2023 구어 말뭉치에서 '싸다' 활용형이 포함된 문장: 1199개
2023 신문 말뭉치에서 '싸다' 활용형이 포함된 문장: 4984개


In [ ]:
if filtered_2022_talk_sentences:
    print("\n📂 2022 구어 말뭉치 예제")
    for i, (id_, sentence) in enumerate(list(filtered_2022_talk_sentences.items())[:30]):  # 처음 10개만 출력
        print(f"{i+1}. ID: {id_} | 문장: {sentence}")

if filtered_2023_talk_sentences:
    print("\n📂 2023 구어 말뭉치 예제")
    for i, (id_, sentence) in enumerate(list(filtered_2023_talk_sentences.items())[:30]):  # 처음 10개만 출력
        print(f"{i+1}. ID: {id_} | 문장: {sentence}")

if filtered_2023_news_sentences:
    print("\n📂 2023 신문 말뭉치 예제")
    for i, (id_, sentence) in enumerate(list(filtered_2023_news_sentences.items())[:30]):  # 처음 10개만 출력
        print(f"{i+1}. ID: {id_} | 문장: {sentence}")


📂 2022 구어 말뭉치 예제
1. ID: SDRW2200001995.1.1.269 | 문장: 도시락 [싸]서 챙겨 주고
2. ID: SDRW2200002020.1.1.222 | 문장: 단백질 보충하기 위해서 소고기 사 먹는 거보다는 [싸]다.
3. ID: SDRW2200002020.1.1.239 | 문장: 아니 굴 자체가 어 우리나라에는 굉장히 [싸]지만은
4. ID: SDRW2200002025.1.1.84 | 문장: 혹시 도시락 [싸] 갖고 다니셨습니까?
5. ID: SDRW2200002025.1.1.85 | 문장: 그럼요 도시락은 당연히 [싸] 가지고 다녀 다녀야죠.
6. ID: SDRW2200002025.1.1.89 | 문장: 그리고 반찬 김치 [싸] 가지고 가면 김치국물
7. ID: SDRW2200002025.1.1.106 | 문장: 어릴 때 도시락을 초등학교 3학년 때까지 [싸]가 보고
8. ID: SDRW2200002028.1.1.132 | 문장: 근데 육개장이 원래 850원이었나? 750원? 엄청 [쌌]잖아
9. ID: SDRW2200002028.1.1.228 | 문장: 이게 [싼] 편인가 비싼 편인가 비교하기도 쉽고
10. ID: SDRW2200002019.1.1.36 | 문장: 조금 더 [싼] 걸로 기다려 보려고 아직 예약 안 했어요.
11. ID: SDRW2200002019.1.1.54 | 문장: 응 괜찮지 회. 응 거 회는 조금 [싸]게 먹을라 그러면은
12. ID: SDRW2200002019.1.1.61 | 문장: 거기에 [싸]서 먹는 횟집이 있다고.
13. ID: SDRW2200002033.1.1.121 | 문장: 아 저 [싼] 걸 먹어라 이게 아니라?
14. ID: SDRW2200002033.1.1.122 | 문장: 비싼 게 맛있는 거고 [싼] 거는 맛없는 거.
15. ID: SDRW2200002033.1.1.219 | 문장: 최 아주 돈 [싼] 돈으로 자기네 그냥 필요한 인력 뽑아 갖고 굴리는 걔네들은
16. ID: SDRW220

# 저장

In [ ]:
import csv
import os
import pandas as pd

# 저장 경로 설정
save_path = "/content/drive/MyDrive/2025-2. 홍콩대 학술교류/필터링 결과"
os.makedirs(save_path, exist_ok=True)

# CSV 및 XLSX 파일 저장 함수
def save_to_csv(filename, data_dict):
    """
    주어진 딕셔너리를 CSV 파일로 저장
    """
    file_path = os.path.join(save_path, filename)
    with open(file_path, mode='w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["ID", "Sentence"])
        for key, sentence in data_dict.items():
            writer.writerow([key, sentence])

def save_to_xlsx(filename, data_dict):
    """
    주어진 딕셔너리를 XLSX 파일로 저장
    """
    file_path = os.path.join(save_path, filename)
    df = pd.DataFrame(list(data_dict.items()), columns=["ID", "Sentence"])
    df.to_excel(file_path, index=False)

In [ ]:
# CSV 및 XLSX 파일로 저장
save_to_csv(f"{WORD_WE_USE}_2022_talk_sentences.csv", filtered_2022_talk_sentences)
save_to_xlsx(f"{WORD_WE_USE}_2022_talk_sentences.xlsx", filtered_2022_talk_sentences)

save_to_csv(f"{WORD_WE_USE}_2023_talk_sentences.csv", filtered_2023_talk_sentences)
save_to_xlsx(f"{WORD_WE_USE}_2023_talk_sentences.xlsx", filtered_2023_talk_sentences)

save_to_csv(f"{WORD_WE_USE}_2023_news_sentences.csv", filtered_2023_news_sentences)
save_to_xlsx(f"{WORD_WE_USE}_2023_news_sentences.xlsx", filtered_2023_news_sentences)

# 결과 출력
print(f"샘플링된 2022 구어 말뭉치에서 '{WORD_WE_USE}' 활용형이 포함된 문장: {len(filtered_2022_talk_sentences)}개")
print(f"샘플링된 2023 구어 말뭉치에서 '{WORD_WE_USE}' 활용형이 포함된 문장: {len(filtered_2023_talk_sentences)}개")
print(f"샘플링된 2023 신문 말뭉치에서 '{WORD_WE_USE}' 활용형이 포함된 문장: {len(filtered_2023_news_sentences)}개")

샘플링된 2022 구어 말뭉치에서 '싸다' 활용형이 포함된 문장: 1692개
샘플링된 2023 구어 말뭉치에서 '싸다' 활용형이 포함된 문장: 1199개
샘플링된 2023 신문 말뭉치에서 '싸다' 활용형이 포함된 문장: 4984개


# 연습

In [ ]:
# 테스트 문장
test_sentences = ["이 물은 차다.",
                  "나는 공을 찼다.",
                  "약간 휴식 취하고 이런 경찰들 좀 있잖아 차 세워 놓고.",
                  "대법원의 판결문 초안이 공개된 다음날인 3일(현지시간) 워싱턴의 연방대법원 앞에는 임신중단 찬·반론자 수백명이 몰려들었다. 찬성론자들은 ‘내 몸에 대한 선택은 내가 한다’ 등의 손팻말을 들고 구호를 외치며 로 대 웨이드 판례를 뒤집으려는 대법원에 항의했다. ‘법원은 당신의 권리에 관심이 없다’는 구호도 등장했다. 반면 천주교 신자 등 임신중단 반대론자들은 “임신중단은 살인”이라며 대법원 결정을 지지했다. 경찰은 대법원 정문 앞 도로를 경찰차와 바리케이드로 막고 물리적 충돌에 대비했다."
                  ]

# 형태소 분석 수행 및 출력
for sentence in test_sentences:
    analyzed = kiwi.analyze(sentence)

    print(analyzed)

    print(f"\n🔹 문장: {sentence}")
    for token, pos, _, _ in analyzed[0][0]:
        print(f"{token} ({pos})")

[([Token(form='이', tag='MM', start=0, len=1), Token(form='물', tag='NNG', start=2, len=1), Token(form='은', tag='JX', start=3, len=1), Token(form='차', tag='VA', start=5, len=1), Token(form='다', tag='EF', start=6, len=1), Token(form='.', tag='SF', start=7, len=1)], -29.898645401000977)]

🔹 문장: 이 물은 차다.
이 (MM)
물 (NNG)
은 (JX)
차 (VA)
다 (EF)
. (SF)
[([Token(form='나', tag='NP', start=0, len=1), Token(form='는', tag='JX', start=1, len=1), Token(form='공', tag='NNG', start=3, len=1), Token(form='을', tag='JKO', start=4, len=1), Token(form='차', tag='VV', start=6, len=1), Token(form='었', tag='EP', start=6, len=1), Token(form='다', tag='EF', start=7, len=1), Token(form='.', tag='SF', start=8, len=1)], -21.22184181213379)]

🔹 문장: 나는 공을 찼다.
나 (NP)
는 (JX)
공 (NNG)
을 (JKO)
차 (VV)
었 (EP)
다 (EF)
. (SF)
[([Token(form='약간', tag='MAG', start=0, len=2), Token(form='휴식', tag='NNG', start=3, len=2), Token(form='취하', tag='VV', start=6, len=2), Token(form='고', tag='EC', start=8, len=1), Token(form='이런', tag='MM', sta